In [1]:
!gpustat

seven                Wed Mar 10 18:02:48 2021  440.64
[0] TITAN RTX        | 34'C,   0 % |    11 / 24220 MB |
[1] TITAN RTX        | 71'C,  37 % |  2662 / 24220 MB | cwb(2651M)
[2] TITAN RTX        | 32'C,   0 % |    11 / 24220 MB |
[3] TITAN RTX        | 86'C,  98 % |  9928 / 24220 MB | cwb(3411M) cwb(3063M) cwb(3443M)


In [9]:
from os.path import join as pathjoin
from data_processing import *
from interpretation import *
from models import *
from training import *

In [10]:
ddd = pd.read_csv('/home/mlepekhin/data/ru_train')
ddd = ddd.append(pd.read_csv('/home/mlepekhin/data/en_train'))
ddd = ddd.append(pd.read_csv('/home/mlepekhin/data/new_ru_attacked_30.csv'))
ddd = ddd.append(pd.read_csv('/home/mlepekhin/data/en_attacked_30.csv')).sample(frac=1, random_state=42)
print(ddd.shape)
ddd.head()

(3867, 7)


,Unnamed: 0,target,text,changed_words_num,new_model_target,old_model_target,old_text
727,141,A1,"A great way to start , I think , with my view ...",NaN,NaN,NaN,NaN
1420,528,A1,Реакция курильщиков на вторую волну антитабачн...,NaN,NaN,NaN,NaN
1031,1049,A9,1 . - ( 1 ) The corporation sole by the name o...,NaN,NaN,NaN,NaN
137,1121,A12,We noticed that you 're using an unsupported b...,NaN,NaN,NaN,NaN
952,1755,A14,ПЕРВЫЕ РЕЗУЛЬТАТЫ ЭКСПЕРИМЕНТА ПО РАДИОЛОКАЦИИ...,NaN,NaN,NaN,NaN


In [11]:
ddd.to_csv('enru_attacked_train_30.csv')

Let's download all the data.

In [12]:
DATA_DIR = '/home/mlepekhin/data'
MODELS_DIR = '/home/mlepekhin/models'
MODEL_ID = 'allennlp_xlm_roberta_enru_attacked_30' 
CHECKPOINTS_DIR = pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints')
BEST_MODEL = pathjoin(CHECKPOINTS_DIR, 'best.th')

In [13]:
transformer_model = 'xlm-roberta-base'
MAX_TOKENS = 512

In [14]:
tokenizer = PretrainedTransformerTokenizer(transformer_model)

03102021 18:05:44|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-config.json from cache at /home/mlepekhin/.cache/torch/transformers/762ddd751172e9d3229e5da17a459eee6c0dfdc237c718944d0b1a85f06c7e1e.2b0f807393c56e8861a31cd67d2fc0b45d71d9735dd47dd66afb650f90b6d2a8
03102021 18:05:44|INFO|transformers.configuration_utils| Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 250002
}

03102021 18:05:44|INFO|transformers.tokenization_

In [15]:
train_dataset_reader = build_transformer_dataset_reader(transformer_model, lower=True)
val_dataset_reader = build_transformer_dataset_reader(transformer_model, lower=True)

train_data, dev_data = read_data(
    "enru_attacked_train_30.csv", 
    pathjoin(DATA_DIR, "manual_genres", "all.csv"),
    train_dataset_reader, 
    val_dataset_reader
)

vocab = build_vocab(train_data + dev_data)

train_data.index_with(vocab)
dev_data.index_with(vocab)

03102021 18:05:52|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-config.json from cache at /home/mlepekhin/.cache/torch/transformers/762ddd751172e9d3229e5da17a459eee6c0dfdc237c718944d0b1a85f06c7e1e.2b0f807393c56e8861a31cd67d2fc0b45d71d9735dd47dd66afb650f90b6d2a8
03102021 18:05:52|INFO|transformers.configuration_utils| Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 250002
}

03102021 18:05:52|INFO|transformers.tokenization_

03102021 18:06:03|INFO|transformers.tokenization_utils| loading file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-sentencepiece.bpe.model from cache at /home/mlepekhin/.cache/torch/transformers/0c370616ddfc06067c0634160f749c2cf9d8da2c50e03a2617ce5841c8df3b1d.309f0c29486cffc28e1e40a2ab0ac8f500c203fe080b95f820aa9cb58e5b84ed
03102021 18:06:04|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-config.json from cache at /home/mlepekhin/.cache/torch/transformers/762ddd751172e9d3229e5da17a459eee6c0dfdc237c718944d0b1a85f06c7e1e.2b0f807393c56e8861a31cd67d2fc0b45d71d9735dd47dd66afb650f90b6d2a8
03102021 18:06:04|INFO|transformers.configuration_utils| Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
 

Reading data
<class 'data_processing.ClassificationDatasetReader'> enru_attacked_train_30.csv


03102021 18:11:34|INFO|allennlp.data.vocabulary| Fitting token dictionary from dataset.



Building the vocabulary


In [16]:
model = build_pool_transformer_model(vocab, transformer_model)

Building the model


03102021 18:11:34|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-config.json from cache at /home/mlepekhin/.cache/torch/transformers/762ddd751172e9d3229e5da17a459eee6c0dfdc237c718944d0b1a85f06c7e1e.2b0f807393c56e8861a31cd67d2fc0b45d71d9735dd47dd66afb650f90b6d2a8
03102021 18:11:34|INFO|transformers.configuration_utils| Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 250002
}

03102021 18:11:35|INFO|transformers.modeling_util

In [17]:
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
print(cuda_device)

0


In [18]:
!rm -rf {CHECKPOINTS_DIR}
!mkdir -p {CHECKPOINTS_DIR}

In [19]:
train_loader, dev_loader = build_data_loaders(train_data, dev_data)

# You obviously won't want to create a temporary file for your training
# results, but for execution in binder for this course, we need to do this.

trainer = build_classifier_trainer(
    model,
    pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints'),
    train_loader,
    dev_loader,
    10,
    cuda_device=cuda_device
)
print("Starting training")
trainer.train()
print("Finished training")

03102021 18:11:52|INFO|allennlp.training.optimizers| Number of trainable parameters: 278052107
03102021 18:11:52|WARNING|allennlp.training.trainer| You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
03102021 18:11:52|INFO|allennlp.training.trainer| Beginning training.
03102021 18:11:52|INFO|allennlp.training.trainer| Epoch 0/9
03102021 18:11:52|INFO|allennlp.training.trainer| Worker 0 memory usage MB: 5927.872


Starting training


03102021 18:11:52|INFO|allennlp.training.trainer| GPU 0 memory usage MB: 1990
03102021 18:11:52|INFO|allennlp.training.trainer| GPU 1 memory usage MB: 2662
03102021 18:11:52|INFO|allennlp.training.trainer| GPU 2 memory usage MB: 11
03102021 18:11:52|INFO|allennlp.training.trainer| GPU 3 memory usage MB: 9928
03102021 18:11:52|INFO|allennlp.training.trainer| Training


03102021 18:14:07|INFO|allennlp.training.trainer| Validating


03102021 18:14:49|INFO|allennlp.training.tensorboard_writer|                        Training |  Validation
03102021 18:14:49|INFO|allennlp.training.tensorboard_writer| accuracy           |     0.684  |     0.715
03102021 18:14:49|INFO|allennlp.training.tensorboard_writer| gpu_0_memory_MB    |  1990.000  |       N/A
03102021 18:14:49|INFO|allennlp.training.tensorboard_writer| gpu_1_memory_MB    |  2662.000  |       N/A
03102021 18:14:49|INFO|allennlp.training.tensorboard_writer| gpu_2_memory_MB    |    11.000  |       N/A
03102021 18:14:49|INFO|allennlp.training.tensorboard_writer| gpu_3_memory_MB    |  9928.000  |       N/A
03102021 18:14:49|INFO|allennlp.training.tensorboard_writer| loss               |     1.007  |     0.861
03102021 18:14:49|INFO|allennlp.training.tensorboard_writer| reg_loss           |     0.000  |     0.000
03102021 18:14:49|INFO|allennlp.training.tensorboard_writer| worker_0_memory_MB |  5927.872  |       N/A


...

03102021 18:41:23|INFO|allennlp.training.trainer| Epoch duration: 0:03:15.130420
03102021 18:41:23|INFO|allennlp.training.trainer| Estimated training time remaining: 0:03:16
03102021 18:41:23|INFO|allennlp.training.trainer| Epoch 9/9
03102021 18:41:23|INFO|allennlp.training.trainer| Worker 0 memory usage MB: 5927.872
03102021 18:41:23|INFO|allennlp.training.trainer| GPU 0 memory usage MB: 18526
03102021 18:41:23|INFO|allennlp.training.trainer| GPU 1 memory usage MB: 2662
03102021 18:41:23|INFO|allennlp.training.trainer| GPU 2 memory usage MB: 18526
03102021 18:41:23|INFO|allennlp.training.trainer| GPU 3 memory usage MB: 9928
03102021 18:41:23|INFO|allennlp.training.trainer| Training


03102021 18:43:50|INFO|allennlp.training.trainer| Validating


03102021 18:44:35|INFO|allennlp.training.tensorboard_writer|                        Training |  Validation
03102021 18:44:35|INFO|allennlp.training.tensorboard_writer| accuracy           |     0.988  |     0.742
03102021 18:44:35|INFO|allennlp.training.tensorboard_writer| gpu_0_memory_MB    |  18526.000  |       N/A
03102021 18:44:35|INFO|allennlp.training.tensorboard_writer| gpu_1_memory_MB    |  2662.000  |       N/A
03102021 18:44:35|INFO|allennlp.training.tensorboard_writer| gpu_2_memory_MB    |  18526.000  |       N/A
03102021 18:44:35|INFO|allennlp.training.tensorboard_writer| gpu_3_memory_MB    |  9928.000  |       N/A
03102021 18:44:35|INFO|allennlp.training.tensorboard_writer| loss               |     0.037  |     1.237
03102021 18:44:35|INFO|allennlp.training.tensorboard_writer| reg_loss           |     0.000  |     0.000
03102021 18:44:35|INFO|allennlp.training.tensorboard_writer| worker_0_memory_MB |  5927.872  |       N/A


03102021 18:44:39|INFO|allennlp.training.trainer| Epoch duration: 0:03:16.404866
03102021 18:44:39|INFO|allennlp.training.checkpointer| loading best weights


Finished training


In [20]:
!rm "{CHECKPOINTS_DIR}"/*.json "{CHECKPOINTS_DIR}"/model* "{CHECKPOINTS_DIR}"/training*
model.vocab.save_to_files(pathjoin(MODELS_DIR, MODEL_ID, 'vocab'))
!ls -lh "{CHECKPOINTS_DIR}"

03102021 18:44:41|WARNING|root| vocabulary serialization directory /home/mlepekhin/models/allennlp_xlm_roberta_enru_attacked_30/vocab is not empty
03102021 18:44:41|INFO|filelock| Lock 140457197761488 acquired on /home/mlepekhin/models/allennlp_xlm_roberta_enru_attacked_30/vocab/.lock
03102021 18:44:42|INFO|filelock| Lock 140457197761488 released on /home/mlepekhin/models/allennlp_xlm_roberta_enru_attacked_30/vocab/.lock


total 1,1G
-rw-rw-r-- 1 mlepekhin mlepekhin 1,1G мар 10 18:28 best.th
drwxrwxr-x 4 mlepekhin mlepekhin 4,0K мар 10 18:11 log
